#### 使用Scipy求解SVM

In [1]:
from scipy.optimize import minimize
import numpy as np
class SVM:
    def __init__(self,C=0.001):
        self.c=C
        self.relaxation_type='ineq'
        if self.c==0:
            self.relaxation_type='eq'
        
    def fit(self,X,y):
        y=2*y-1
        m,n=X.shape
        unknow_vector=np.ones(m+n+1)/(m+n+1)
        
        self.solver=minimize(type(self).minimize_object,unknow_vector,args=(X,y,self.c),
              constraints=({'type': 'ineq', 'args': (X,y),
                            'fun':lambda unknow_vector,X,y:type(self).linear_onstraint(unknow_vector,X,y)},
                           {'type':self.relaxation_type, 'args': (y,),'fun':lambda unknow_vector,y:type(self).relaxation_constraint(unknow_vector,y)}                         
             ))
        self.coef_=self.solver.x[:n]
        self.intercept_=b=self.solver.x[n]
        self.beta_=self.solver.x[n+1:]
        
        self.support_value=y*(np.dot(X,self.coef_.T)+self.intercept_)+self.beta_
        self.support_=np.where(self.support_value<=1.01)
        self.support_vectors_=X[self.support_]
           
        return self
    
    def predict(self,X):
        pred=X@self.coef_+self.intercept_
        return (np.sign(pred)+1)/2
        
    
    
    def minimize_object(unknow_vector,X,y,c):
        m,n=X.shape
        w=unknow_vector[:n]  
        b=unknow_vector[n]   
        beta=unknow_vector[n+1:]
        object_value=0.5*np.sum(w**2)+c*np.sum(beta)
        return object_value

    def linear_onstraint(unknow_vector,X,y):
        m,n=X.shape
        w=unknow_vector[:n]
        b=unknow_vector[n]
        beta=unknow_vector[n+1:]
        return y*(X@w.T+b)+beta-1

    def relaxation_constraint(unknow_vector,y):
        n=len(y)
        beta=unknow_vector[n+1:]    
        return beta

In [5]:
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X,y=load_breast_cancer().data,load_breast_cancer().target
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
my_svm=SVM(C=0.).fit(X_train,y_train)
pred=my_svm.predict(X_test)
pred

array([1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1.,
       1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0.,
       1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0.,
       1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1.,
       0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0.,
       0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1.,
       0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1.,
       1.])

In [6]:
accuracy_score(y_test,pred)

0.8771929824561403

In [7]:
my_svm.support_vectors_

array([[2.811e+01, 1.847e+01, 1.885e+02, 2.499e+03, 1.142e-01, 1.516e-01,
        3.201e-01, 1.595e-01, 1.648e-01, 5.525e-02, 2.873e+00, 1.476e+00,
        2.198e+01, 5.256e+02, 1.345e-02, 2.772e-02, 6.389e-02, 1.407e-02,
        4.783e-02, 4.476e-03, 2.811e+01, 1.847e+01, 1.885e+02, 2.499e+03,
        1.142e-01, 1.516e-01, 3.201e-01, 1.595e-01, 1.648e-01, 5.525e-02],
       [1.917e+01, 2.480e+01, 1.324e+02, 1.123e+03, 9.740e-02, 2.458e-01,
        2.065e-01, 1.118e-01, 2.397e-01, 7.800e-02, 9.555e-01, 3.568e+00,
        1.107e+01, 1.162e+02, 3.139e-03, 8.297e-02, 8.890e-02, 4.090e-02,
        4.484e-02, 1.284e-02, 2.096e+01, 2.994e+01, 1.517e+02, 1.332e+03,
        1.037e-01, 3.903e-01, 3.639e-01, 1.767e-01, 3.176e-01, 1.023e-01],
       [1.791e+01, 2.102e+01, 1.244e+02, 9.940e+02, 1.230e-01, 2.576e-01,
        3.189e-01, 1.198e-01, 2.113e-01, 7.115e-02, 4.030e-01, 7.747e-01,
        3.123e+00, 4.151e+01, 7.159e-03, 3.718e-02, 6.165e-02, 1.051e-02,
        1.591e-02, 5.099e-03, 2.080e